In [16]:
from dotenv import load_dotenv
import os

load_dotenv()  # Lit le fichier .env et charge les variables

# On récupère la variable en utilisant son NOM
api_key = os.getenv("MISTRAL_API_KEY") 


👉 Exercice 3.1 — En adaptant le script, affichez également response.usage_metadata afin d'inspecter le nombre de tokens consommés. Calculez ensuite un coût approximatif en vous basant sur le tarif public (input_tokens et output_tokens).

In [17]:
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_core.messages import HumanMessage

llm = ChatMistralAI(
    model="ministral-3b-2410",
    temperature=0,
    api_key=api_key,
)

message = HumanMessage(content="Quelle est la capitale de l'Albanie ?")
response = llm.invoke([message])

print(response.content)
print(response.usage_metadata)

ImportError: cannot import name 'Sentinel' from 'typing_extensions' (C:\Users\Gamer\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\typing_extensions.py)

In [7]:
# ---- PARAMÈTRES ----
prix_input = 0.0005   # prix par 1k input tokens (exemple)
prix_output = 0.0015  # prix par 1k output tokens (exemple)

# ---- VALEURS ----
input_tokens = 12000
output_tokens = 8000

# ---- CALCUL ----
cout = (input_tokens/1000) * prix_input + (output_tokens/1000) * prix_output

print(f"Coût total approximatif : {cout:.4f} $")


Coût total approximatif : 0.0180 $


👉 Exercice 3.2 — Modifiez la température (temperature=0.7) et observez les différences de réponses pour une requête créative.

In [11]:
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_core.messages import HumanMessage

llm = ChatMistralAI(
    model="ministral-3b-2410",
    temperature=0.7)

message = HumanMessage(content="Quelle est la capitale de l'Albanie ?")
response = llm.invoke([message])

print(response.content)
print(response.usage_metadata)

La capitale de l'Albanie est Tirana.
{'input_tokens': 14, 'output_tokens': 12, 'total_tokens': 26}


Le fait de changer la température n'a rien changé à la réponse. Même si on augmente jusqu'a 1 

In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_mistralai.chat_models import ChatMistralAI

model = ChatMistralAI(model="ministral-3b-2410", temperature=0)

prompt = ChatPromptTemplate.from_template(
    "Fais-moi une blague sur le sujet : {sujet}"
)
output_parser = StrOutputParser()

chain = prompt | model | output_parser

for sujet in ["pompier", "police"]:
    print(chain.invoke({"sujet": sujet}))
    print("-" * 10)

Pourquoi les pompiers ne jouent-ils jamais au poker ? Parce qu'ils ne veulent pas que les cartes soient en feu !
----------
Pourquoi les policiers ne jouent-ils jamais au poker ? Parce qu'ils ne veulent pas se faire "carré" !
----------


👉 Exercice 4.1 — Remplacez la liste de sujets par une liste saisie dynamiquement (prompt utilisateur ou lecture d'un fichier). Ajoutez un contrôle qui empêche l'envoi d'une chaîne vide.

In [18]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_mistralai.chat_models import ChatMistralAI

model = ChatMistralAI(model="ministral-3b-2410", temperature=0)

prompt = ChatPromptTemplate.from_template(
    "Fais-moi une blague sur le sujet : {sujet}"
)
output_parser = StrOutputParser()

chain = prompt | model | output_parser

# ✅ Saisie dynamique
sujets_input = input("Entrez une liste de sujets séparés par des virgules : ")
sujets = [s.strip() for s in sujets_input.split(",")]

for sujet in sujets:
    if not sujet:   
        print("⛔ Sujet vide ignoré.")
        continue

    print(chain.invoke({"sujet": sujet}))
    print("-" * 10)


Pourquoi les feux de camp aiment-ils les blagues ?

- Parce qu'ils aiment les "feu" de rire !
----------
Pourquoi les chauves sont-elles mauvais en mathématiques ? Parce qu'elles ont toujours des problèmes de "calculs" !
----------


👉 Exercice 4.2 — Créez une variante de la chaîne produisant des réponses au format JSON (par exemple avec les clés setup et punchline). Pour cela, adaptez le prompt et utilisez StrOutputParser() ou un parser JSON.

In [19]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_mistralai.chat_models import ChatMistralAI

model = ChatMistralAI(model="ministral-3b-2410", temperature=0)

prompt_json = ChatPromptTemplate.from_template(
    """
    Crée une blague sur le sujet : {sujet}
    Renvoie le résultat uniquement au format JSON :

    {{
        "setup": "Texte d’introduction",
        "punchline": "Chute"
    }}
    """
)

output_parser = StrOutputParser()
chain_json = prompt_json | model | output_parser

print(chain_json.invoke({"sujet": "police"}))


{
    "setup": "Pourquoi les policiers ne jouent-ils jamais au poker ?",
    "punchline": "Parce qu'ils ne veulent pas se faire 'coupé'"
}


👉 Exercice 4.3 — Ajoutez un paramètre temperature passé dynamiquement via chain.invoke pour comparer l'impact sur plusieurs sujets.